Run the following cells in the provided order to see the statistics for the Project / Experiment / Iteration combination

Before running the code, make sure you have the following variables defined in .cabconfig file as per your MongoDB instance:
- password
- username
- host
- port

Then, in the second sell, define your project_name, experiment_name and iteration_name

In [ ]:
# Import libraries and connect to the database

import os, sys
sys.path.append(os.path.dirname(os.path.abspath('..')))
import cabutils
db_connection = cabutils.get_db_connection()
print("Connected to database" if db_connection else "Connection failed")

In [ ]:
# Define your project, experiment and iteration names below before running this cell

project_name = "OCP"
experiment_name = "physion"
iteration_name = "1"

In [ ]:
# This cell processes the information from the database and calculates general statistics for the given project, experiment and iteration

sequence_collection = db_connection[f"{project_name}_input"][experiment_name]
results_collection = db_connection[f"{project_name}_output"][experiment_name]
sequences = sequence_collection.find({"iteration": iteration_name}, {"_id": 1, "games": 1, "familiarization_stims": 1, "stims": 1})
games_played_stats = dict()
games_completed_stats = dict()
for sequence in sequences:
    # Calculate number of played games
    games_played = len(sequence.get("games", []))
    games_played_stats.update({games_played: 1 + games_played_stats.get(games_played, 0)})

    # Calculate number of completed games
    games_completed = 0
    game_stimuli = list(filter(lambda x: x is not None, [stim.get("stimulus_name") for stim in sequence.get('stims', {}).values()]))

    for game_id in sequence.get("games", []):
        stimuli_played = filter(lambda x: x is not None, [game.get("stimulus_name") for game in results_collection.find({"gameID": game_id}, {"stimulus_name": 1})])
        game_stims = game_stimuli.copy()
        for stim in stimuli_played:
            if stim in game_stims:
                game_stims.remove(stim)
        games_completed += (1 if len(game_stims) == 0 else 0)
    games_completed_stats.update({games_completed: 1 + games_completed_stats.get(games_completed, 0)})

games_played_stats_list = [["Number of games played", "Qty of session templates"]]
for key, value in sorted(games_played_stats.items(), key=lambda x: x[0]):
    games_played_stats_list.append([key, value])

games_completed_stats_list = [["Number of games completed", "Qty of session templates"]]
for key, value in sorted(games_completed_stats.items(), key=lambda x: x[0]):
    games_completed_stats_list.append([key, value])

In [ ]:
# This cell renders the statistics in a table and a chart format

from IPython.display import HTML, display
import pandas as pd
import plotly.express as px

def render_table(rows):
    return '<table><tr>{}</tr></table>'.format('</tr><tr>'.join('<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in rows))

def render_chart(rows):
    df = pd.DataFrame(rows)
    fig = px.bar(df, x=0, y=1)
    fig.update_layout(height=600, title_text='Number of session templates by games played')
    fig.show()

display(HTML(f"<h2>Project {project_name} | Experiment {experiment_name} | Iteration {iteration_name}</h2><h3>Statistics</h3>"))
display(HTML(render_table(games_played_stats_list)))
render_chart(games_played_stats_list[1:])

display(HTML(render_table(games_completed_stats_list)))
render_chart(games_completed_stats_list[1:])